In [1]:
import pretty_midi as pm
# import soundfile as sf
import os
import music21 as m21

d:\GitHub\IntelOb\.music\Lib\site-packages\pretty_midi\instrument.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
def pretty_midi_to_music21(pm: pm.PrettyMIDI) -> m21.stream.Score:
    score = m21.stream.Score()
    part_streams = []

    # Use first time signature and tempo as defaults
    ts_changes = pm.time_signature_changes
    tempo_changes, tempi = pm.get_tempo_changes()

    global_ts = ts_changes[0] if ts_changes else None
    global_tempo = tempi[0] if len(tempi) > 0 else 120

    for inst in pm.instruments:
        part = m21.stream.Part()

        # Set part name
        if inst.name:
            part.id = inst.name
        else:
            inst_name = pm.program_to_instrument_name(inst.program)
            part.id = inst_name

        # Set instrument
        if inst.is_drum:
            m21_inst = m21.instrument.UnpitchedPercussion()
        else:
            # Try to get instrument by name
            try:
                inst_name = pm.program_to_instrument_name(inst.program)
                m21_inst = m21.instrument.fromString(inst_name)
            except:
                m21_inst = m21.instrument.Instrument()

        part.append(m21_inst)

        # Add global time signature and tempo
        if global_ts:
            ts = m21.meter.TimeSignature(f"{global_ts.numerator}/{global_ts.denominator}")
            part.append(ts)

        part.append(m21.tempo.MetronomeMark(number=global_tempo))

        for n in inst.notes:
            m21_note = m21.note.Note(n.pitch)
            m21_note.quarterLength = n.end - n.start
            m21_note.offset = n.start
            m21_note.volume.velocity = n.velocity
            part.append(m21_note)

        part_streams.append(part)

    for p in part_streams:
        score.append(p)

    return score


In [3]:
os.listdir(".")

['GeneticAlg', 'RGR.ipynb', 'RGR.py', 'RGR_.ipynb']

In [5]:
SAMPLING_RATE = 44100
SECONDS = 15
filename = "../Music/Beethoven-Moonlight-Sonata"
midi_file = filename + ".mid"
mp3_file = filename + ".mp3"
midi = pm.PrettyMIDI(midi_file)
# midi = pretty_midi_to_music21(midi)

In [13]:
print(midi.instruments[0].name)
# dir(midi.instruments[0].program)
# 'duration',
# 'end',
# 'get_duration',
# 'pitch',
# 'start',
# 'velocity']

# 'control_changes',
# 'fluidsynth',
# 'get_chroma',
# 'get_end_time',
# 'get_onsets',
# 'get_piano_roll',
# 'get_pitch_class_histogram',
# 'get_pitch_class_transition_matrix',
# 'is_drum',
# 'name',
# 'notes',
# 'pitch_bends',
# 'program',
# 'remove_invalid_notes',
# 'synthesize']

1st Mvmt Sonata No.14, Opus 27, No.2


In [ ]:
# waveform = midi.fluidsynth(fs=SAMPLING_RATE)
# waveform_short = waveform[:SECONDS*SAMPLING_RATE]
# sf.write(filename, waveform_short, SAMPLING_RATE)
# print(f"[!] Music saved succesfully to {filename}")